In [1]:
import numpy as np
from wnmf import wNMF

In [8]:
## An example on simulated data
n=100
features = 100
components=25
X = 100*np.random.uniform(size=n*features).reshape(features,n)
W = np.ones_like(X)

In [9]:
model = wNMF(n_components=components,beta_loss='kullback-leibler',max_iter=1000,track_error=True,verbose=1)
fit = model.fit(X=X,W=W,n_run=5)

Beginning Run 1...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 2...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 3...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 4...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 5...
|--- Initializing U,V
|--- Running wNMF
|--- Completed


In [10]:
wm = np.load('Small_Weight_Matrix.npy')
dm = np.load('Small_Data_Matrix.npy')
model = wNMF(n_components=components,beta_loss='kullback-leibler',max_iter=1000)
fit = model.fit(X=dm,W=wm,n_run=1)

Beginning Run 1...
|--- Initializing U,V
|--- Running wNMF
|--- Completed


In [ ]:
print(fit.V)

In [ ]:
print(fit.U)

In [ ]:
print(fit.err)

In [ ]:
## testing out altair for faceting / data vis
import pandas as pd
errdf = pd.DataFrame(np.log10(fit.err_tracker)).T.rename({i:'run_'+str(i) for i in range(0,components)},axis='columns')

In [ ]:
import altair as alt
alt.renderers.enable('notebook')
alt.Chart(errdf.loc[100:,:].reset_index().melt(id_vars='index')).mark_point().encode(
    alt.X('index:Q'),
    alt.Y('value:Q',scale=alt.Scale(zero=False)),
    color='variable'
    
)

In [ ]:
## Dead simple euclidean NMF
def nmf_euc(A,U,V,iters=1000):
    err_stored = np.zeros(iters)
    
    erf_V = np.ones_like(V,dtype='float64')*np.finfo(np.float64).eps
    erf_U = np.ones_like(U,dtype='float64')*np.finfo(np.float64).eps

    for i in range(0,iters):
        if i % 100 == 0:
            print(i)

#         err_calc = (A - U @ V)
#         err_stored[i] = (np.trace(err_calc @ err_calc.T))/1000
            
        #V[V==0]=0.00000001
        #U[U==0]=0.00000001
        V = V*((U.T @ A)/((U.T @ U @ V) + erf_V))
        U = U*((A @ V.T)/((U @ V @ V.T) + erf_U))
    
    return({'u': U,'v': V,'err': err_stored})